### Problem set 11
### Submitted by: Abu Huraira

In [18]:
(require minikanren)
(require minikanren/matche)

Consider the following knowledge base:

In [19]:
(defrel (studento name group)
    (matche (cons name group)
        [(alisa . 2)]
        [(bob . 1)]
        [(chloe . 2)]
        [(denise . 1)]
        [(edward . 2)]))

(defrel (friendo name1 name2)
    (matche (cons name1 name2)
        [(alisa . bob)]
        [(alisa . denise)]
        [(bob . chloe)]
        [(bob . edward)]
        [(chloe . denise)]
        [(denise . edward)]))

(defrel (parento parent-name child-name)
    (matche (cons parent-name child-name)
        [(marjorie . bart)]
        [(marjorie . lisa)]
        [(marjorie . maggie)]
        [(homer . bart)]
        [(homer . lisa)]
        [(homer . maggie)]
        [(abraham . homer)]
        [(mona . homer)]
        [(jacqueline . marjorie)]
        [(jacqueline . patty)]
        [(jacqueline . selma)]
        [(clancy . marjorie)]
        [(clancy . patty)]
        [(clancy . selma)]
        [(selma . ling)]))

(defrel (unaryo n)
    (conde
    [(== 'z n)]
    [(fresh (m)
        (== `(s ,m) n)
        (unaryo m))]))

1. Write down and explain the results of the following queries:

    (a) `(run* (y z) (friendo alisa y) (friendo y z))`

In [20]:
(run* (y z) (friendo 'alisa y) (friendo y z))

'((bob chloe) (bob edward) (denise edward))

**Explanation:** The result is `'((bob chloe) (bob edward) (denise edward))`
    
The query looks for pairs `(y, z)` where `y` is a friend of Alisa and `z` is a friend of `y`. The result will be a list of all such pairs. 

(b) `(run* (friendo x y))`

In [21]:
(run* (x y) (friendo x y))

'((alisa bob) (alisa denise) (bob chloe) (bob edward) (chloe denise) (denise edward))

**Explanation:** This query looks for all pairs of friends in the knowledge base. The result will be a list of all pairs `(x, y)` where `x` is a friend of `y`.

(c) `(run* (y) (parento jacqueline y) (parento y ling))`

In [22]:
(run* (y) (parento 'jacqueline y) (parento y 'ling))

'(selma)

**Explanation:** Selma is a parent of Ling and Jacqueline is a parent of Selma. 

2. Write down relation groupmateso that checks whether two students are from the same group.
```racket
    (run 1 (q) (groupmateso 'alisa 'bob)) ; '()
    (run 1 (q) (groupmateso 'alisa 'edward)) ; '(_.0)
```

**Solution:**

In [23]:
(defrel (groupmateso student1 student2)
    (fresh (group)
    (conde
    [(studento student1 group)
     (studento student2 group)])))


(displayln (run 1 (q) (groupmateso 'alisa 'bob)))

(displayln (run 1 (q) (groupmateso 'alisa 'edward)))

()
(_.0)


3. Implement predicate `relativeo` that checks whether two people are related by blood (share a common ancestor):

```racket
    (run 1 (q) (relativeo 'selma 'patty)) ; '(_.0)
    (run 1 (q) (relativeo 'lisa 'ling)) ; '(_.0)
    (run 1 (q) (relativeo 'lisa 'selma)) ; '(_.0)
    (run 1 (q) (relativeo 'homer 'selma)) ; '()
```

**Solution:**

In [24]:
; First let's define ancestoro function without creating infinitely deep search tree. 
(defrel (ancestoro name descendant)
    (conde
    [(parento name descendant)]
    [(fresh (child)
        (parento name child)
        (ancestoro child descendant))]))

; Secondly, let's define relativeo function using ancestoro function.
(defrel (relativeo person1 person2)
    (fresh (ancestor)
    (conde
        [(ancestoro ancestor person1)
        (ancestoro ancestor person2)])))

(displayln (run 1 (q) (relativeo 'selma 'patty)))
(displayln (run 1 (q) (relativeo 'lisa 'ling)))
(displayln (run 1 (q) (relativeo 'lisa 'selma)))
(displayln (run 1 (q) (relativeo 'homer 'selma)))

(_.0)
(_.0)
(_.0)
()


4. Implement the following predicates for unary numbers:

(a) Implement a predicate `doubleo` that checks if first number is exactly two times the second:
```racket
    (run 1 (q) (doubleo '(s (s z)) '(s (s (s (s z)))))) ; '(_.0)
    (run 1 (x) (doubleo '(s (s z)) x)) ; '((s (s (s (s z)))))
    (run 1 (x) (doubleo x '(s (s (s (s z)))))) ; '((s (s z)))
    (run 1 (x) (doubleo x '(s (s (s z))))) ; '()
```

(b) Implement a predicate `leqo` that checks if the first number is less than or equal to the second numbers:
```racket
    (run 1 (q) (leqo '(s (s z)) '(s (s (s (z)))))) ; '(_.0)
    (run 1 (q) (leqo '(s (s (s (s z)))) '(s (s (s z))))) ; '()
```

(c) Implement multiplication for unary numbers as a predicate `multo`:
```racket
    (run 1 (x) (multo '(s (s z)) '(s (s (s z))) x)) ; '((s (s (s (s (s (s z)))))))
    (run 1 (x) (multo x '(s (s (s z))) '(s (s (s (s (s (s (z))))))))) ; '((s (s z)))
```

In [25]:
(defrel (doubleo num1 num2)
    (conde
    [(== 'z num1) (== 'z num2)]
    [(fresh (m1 m2)
        (== `(s ,m1) num1)
        (== `(s (s ,m2)) num2)
        (doubleo m1 m2))]))

(displayln "doubleo testing...")
(displayln (run 1 (q) (doubleo '(s (s z)) '(s (s (s (s z))))))) ; '(_.0)
(displayln (run 1 (x) (doubleo '(s (s z)) x))) ; '((s (s (s (s z)))))
(displayln (run 1 (x) (doubleo x '(s (s (s (s z))))))) ; '((s (s z)))
(displayln (run 1 (x) (doubleo x '(s (s (s z)))))) ; '()

doubleo testing...
(_.0)
((s (s (s (s z)))))
((s (s z)))
()


In [26]:
(defrel (leqo num1 num2)
  (conde
    [(== num1 'z)] 
    [(fresh (m1 m2)
       (== num1 `(s ,m1))
       (== num2 `(s ,m2))
       (leqo m1 m2))]))


(displayln "leqo testing...")
(displayln (run 1 (q) (leqo '(s (s z)) '(s (s (s z)))))) ; '(_.0)
(displayln (run 1 (q) (leqo '(s (s (s (s z)))) '(s (s (s z)))))) ; '()


leqo testing...
(_.0)
()


In [28]:
; First let's define the addo function
(defrel (addo a b c)
  (conde
    [(== 'z a) (== b c)]
    [(fresh (d e)
      (== `(s ,d) a)
      (== `(s ,e) c)
      (addo d b e))]))


; Secondly, let's define the multo function which utilizes the addo function
(defrel (multo a b c)
  (conde
    [(== 'z a) (== 'z c)]
    [(fresh (d e)
      (== `(s ,d) a)
      (addo b e c)
      (multo d b e))]))


(displayln "multo testing...\n")
(displayln (run 1 (x) (multo '(s (s z)) '(s (s (s z))) x))) ; '((s (s (s (s (s (s z)))))))
(displayln (run 1 (x) (multo x '(s (s (s z))) '(s (s (s (s (s (s z))))))))) ; '((s (s z)))

multo testing...

((s (s (s (s (s (s z)))))))
((s (s z)))
